In [29]:
%cd /media/eel/DATAUbuntu/Projetos/Pub2023-2024/PortOpt/

/media/eel/DATAUbuntu/Projetos/Pub2023-2024/PortOpt



Módulos

In [30]:
#Cotações do Yahoo Finance
import yfinance as yf

# Tratamento de Dados
import pandas as pd
from pandas.tseries.offsets import BDay
pd.set_option("display.max_colwidth", 150)

#Gráficos
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns

#Utilidades
from datetime import date
import random

Funções

In [31]:

def busca_informes_diarios_cvm_por_periodo(data_inicio, data_fim):
  datas = pd.date_range(data_inicio, data_fim, freq='MS') 
  informe_completo = pd.DataFrame()

  for data in datas:
    try:
      url ='inf_diario_fi_tratado_{}{:02d}.csv'.format(data.year, data.month)
      informe_mensal = pd.read_csv(url, sep=';', encoding = 'ISO-8859-1')     
    
    except: 
      print("Arquivo {} não encontrado!".format(url))    

    informe_completo = pd.concat([informe_completo, informe_mensal], ignore_index=True)
    
    #informe_completo = informe_completo[informe_completo['SIT'] == 'EM FUNCIONAMENTO NORMAL']
    informe_completo = informe_completo.drop(['Unnamed: 0'],axis=1) 

  return informe_completo

In [32]:
def melhores_e_piores(informes, cadastro, top=5, minimo_de_cotistas=100, classe=''):
  
  cadastro      = cadastro[cadastro['SIT'] == 'EM FUNCIONAMENTO NORMAL']
  fundos        = informes[informes['NR_COTST'] >= minimo_de_cotistas]
  cnpj_informes = fundos['CNPJ_FUNDO'].drop_duplicates()
  
  ##fundos = fundos.pivot(index='DT_COMPTC', columns='CNPJ_FUNDO')
  
  fundos = fundos.pivot_table(
    index='DT_COMPTC',
    columns='CNPJ_FUNDO',
    aggfunc='sum'
)
   
  cotas_normalizadas = fundos['VL_QUOTA'] / fundos['VL_QUOTA'].iloc[0]
  
  if classe == 'multimercado':
    cnpj_cadastro      = cadastro[cadastro['CLASSE'] == 'Fundo Multimercado']['CNPJ_FUNDO']   
    cotas_normalizadas = cotas_normalizadas[cnpj_cadastro[cnpj_cadastro.isin(cnpj_informes)]]

  if classe == 'acoes':
    cnpj_cadastro      = cadastro[cadastro['CLASSE'] == 'Fundo de Ações']['CNPJ_FUNDO']   
    cotas_normalizadas = cotas_normalizadas[cnpj_cadastro[cnpj_cadastro.isin(cnpj_informes)]]

  if classe == 'rendafixa':
    cnpj_cadastro      = cadastro[cadastro['CLASSE'] == 'Fundo de Renda Fixa']['CNPJ_FUNDO']   
    cotas_normalizadas = cotas_normalizadas[cnpj_cadastro[cnpj_cadastro.isin(cnpj_informes)]]

  if classe == 'cambial':
    cnpj_cadastro      = cadastro[cadastro['CLASSE'] == 'Fundo Cambial']['CNPJ_FUNDO']   
    cotas_normalizadas = cotas_normalizadas[cnpj_cadastro[cnpj_cadastro.isin(cnpj_informes)]]
  
  #melhores
  melhores = pd.DataFrame()
  melhores['retorno(%)'] = (cotas_normalizadas.iloc[-1].sort_values(ascending=False)[:top] - 1) * 100
  for cnpj in melhores.index:
    fundo = cadastro[cadastro['CNPJ_FUNDO'] == cnpj]
    melhores.at[cnpj, 'Fundo de Investimento'] = fundo['DENOM_SOCIAL'].values[0]
    melhores.at[cnpj, 'Classe']                = fundo['CLASSE'].values[0]
    melhores.at[cnpj, 'PL']                    = fundo['VL_PATRIM_LIQ'].values[0]

  #piores
  piores = pd.DataFrame()
  piores['retorno(%)'] = (cotas_normalizadas.iloc[-1].sort_values(ascending=True)[:top] - 1) * 100
  for cnpj in piores.index:
    fundo = cadastro[cadastro['CNPJ_FUNDO'] == cnpj]
    piores.at[cnpj, 'Fundo de Investimento'] = fundo['DENOM_SOCIAL'].values[0]
    piores.at[cnpj, 'Classe']                = fundo['CLASSE'].values[0]
    piores.at[cnpj, 'PL']                    = fundo['VL_PATRIM_LIQ'].values[0]
  
  return melhores, piores

In [33]:
def consulta_fundo(informes, cnpj):  
  fundo = informes[informes['CNPJ_FUNDO'] == cnpj].copy()
  fundo.set_index('DT_COMPTC', inplace=True)
  fundo['cotas_normalizadas'] = (fundo['VL_QUOTA'] / fundo['VL_QUOTA'].iloc[0])*100
  return fundo

In [34]:
def ranking_fundos(informes, cadastro, top=5, minimo_de_cotistas=100, classe=''):
  
  cadastro      = cadastro[cadastro['SIT'] == 'EM FUNCIONAMENTO NORMAL']
  fundos        = informes[informes['NR_COTST'] >= minimo_de_cotistas]
  cnpj_informes = fundos['CNPJ_FUNDO'].drop_duplicates()
  
  ##fundos = fundos.pivot(index='DT_COMPTC', columns='CNPJ_FUNDO')
  
  fundos = fundos.pivot_table(
    index='DT_COMPTC',
    columns='CNPJ_FUNDO',
    aggfunc='sum'
)
   
  cotas_normalizadas = fundos['VL_QUOTA'] / fundos['VL_QUOTA'].iloc[0]
  
  if classe == 'multimercado':
    cnpj_cadastro      = cadastro[cadastro['CLASSE'] == 'Fundo Multimercado']['CNPJ_FUNDO']   
    cotas_normalizadas = cotas_normalizadas[cnpj_cadastro[cnpj_cadastro.isin(cnpj_informes)]]

  if classe == 'acoes':
    cnpj_cadastro      = cadastro[cadastro['CLASSE'] == 'Fundo de Ações']['CNPJ_FUNDO']   
    cotas_normalizadas = cotas_normalizadas[cnpj_cadastro[cnpj_cadastro.isin(cnpj_informes)]]

  if classe == 'rendafixa':
    cnpj_cadastro      = cadastro[cadastro['CLASSE'] == 'Fundo de Renda Fixa']['CNPJ_FUNDO']   
    cotas_normalizadas = cotas_normalizadas[cnpj_cadastro[cnpj_cadastro.isin(cnpj_informes)]]

  if classe == 'cambial':
    cnpj_cadastro      = cadastro[cadastro['CLASSE'] == 'Fundo Cambial']['CNPJ_FUNDO']   
    cotas_normalizadas = cotas_normalizadas[cnpj_cadastro[cnpj_cadastro.isin(cnpj_informes)]]
  
  #melhores
  melhores = pd.DataFrame()
  melhores['retorno(%)'] = (cotas_normalizadas.iloc[-1].sort_values(ascending=False)[:top] - 1) * 100
  for cnpj in melhores.index:
    fundo = cadastro[cadastro['CNPJ_FUNDO'] == cnpj]
    melhores.at[cnpj, 'Fundo de Investimento'] = fundo['DENOM_SOCIAL'].values[0]
    melhores.at[cnpj, 'Classe']                = fundo['CLASSE'].values[0]
    melhores.at[cnpj, 'PL']                    = fundo['VL_PATRIM_LIQ'].values[0]
  
  return melhores

In [35]:
def retorno(informes, cadastro, top=5, minimo_de_cotistas=100, classe=''):
  
  cadastro      = cadastro[cadastro['SIT'] == 'EM FUNCIONAMENTO NORMAL']
  fundos        = informes[informes['NR_COTST'] >= minimo_de_cotistas]
  cnpj_informes = fundos['CNPJ_FUNDO'].drop_duplicates()
  
  ##fundos = fundos.pivot(index='DT_COMPTC', columns='CNPJ_FUNDO')
  
  fundos = fundos.pivot_table(
    index='DT_COMPTC',
    columns='CNPJ_FUNDO',
    aggfunc='sum'
)
   
  cotas_normalizadas = fundos['VL_QUOTA'] / fundos['VL_QUOTA'].iloc[0]
  
  if classe == 'multimercado':
    cnpj_cadastro      = cadastro[cadastro['CLASSE'] == 'Fundo Multimercado']['CNPJ_FUNDO']   
    cotas_normalizadas = cotas_normalizadas[cnpj_cadastro[cnpj_cadastro.isin(cnpj_informes)]]

  if classe == 'acoes':
    cnpj_cadastro      = cadastro[cadastro['CLASSE'] == 'Fundo de Ações']['CNPJ_FUNDO']   
    cotas_normalizadas = cotas_normalizadas[cnpj_cadastro[cnpj_cadastro.isin(cnpj_informes)]]

  if classe == 'rendafixa':
    cnpj_cadastro      = cadastro[cadastro['CLASSE'] == 'Fundo de Renda Fixa']['CNPJ_FUNDO']   
    cotas_normalizadas = cotas_normalizadas[cnpj_cadastro[cnpj_cadastro.isin(cnpj_informes)]]

  if classe == 'cambial':
    cnpj_cadastro      = cadastro[cadastro['CLASSE'] == 'Fundo Cambial']['CNPJ_FUNDO']   
    cotas_normalizadas = cotas_normalizadas[cnpj_cadastro[cnpj_cadastro.isin(cnpj_informes)]]
  
  #melhores
  melhores = pd.DataFrame()
  melhores['retorno(%)'] = (cotas_normalizadas.iloc[-1].sort_values(ascending=False)[:top] - 1) * 100
  for cnpj in melhores.index:
    fundo = cadastro[cadastro['CNPJ_FUNDO'] == cnpj]
    melhores.at[cnpj, 'Fundo de Investimento'] = fundo['DENOM_SOCIAL'].values[0]
    melhores.at[cnpj, 'Classe']                = fundo['CLASSE'].values[0]
    melhores.at[cnpj, 'PL']                    = fundo['VL_PATRIM_LIQ'].values[0]
  
  return melhores

In [36]:
def cdi_acumulado(data_inicio, data_fim):
  codigo_bcb = 12
  
  url = 'http://api.bcb.gov.br/dados/serie/bcdata.sgs.{}/dados?formato=json'.format(codigo_bcb)
  cdi = pd.read_json(url)
  cdi['data'] = pd.to_datetime(cdi['data'], dayfirst=True)
  cdi.set_index('data', inplace=True) 
  
  cdi_acumulado = (1 + cdi[data_inicio : data_fim] / 100).cumprod()
  cdi_acumulado.iloc[0] = 1
  return cdi_acumulado

Benckmarks

In [37]:
cdi = cdi_acumulado('2022-01-01', '2024-01-31') * 100
cdi

,valor
data,
2022-01-03,100.000000
2022-01-04,100.069510
2022-01-05,100.104283
2022-01-06,100.139068
2022-01-07,100.173866
...,...
2024-01-25,128.050103
2024-01-26,128.106111
2024-01-29,128.162143


In [38]:
ibov = yf.download('^BVSP', start='2022-01-01', end='2024-01-31')['Adj Close']
ibov = (ibov / ibov.iloc[0])*100
ibov

[*********************100%%**********************]  1 of 1 completed


Date
2022-01-03    100.000000
2022-01-04     99.607398
2022-01-05     97.194049
2022-01-06     97.728104
2022-01-07     98.842401
                 ...    
2024-01-24    122.992244
2024-01-25    123.331922
2024-01-26    124.099806
2024-01-29    123.653317
2024-01-30    122.593868
Name: Adj Close, Length: 519, dtype: float64


Buscando Dados da CVM

In [39]:
cadastro = pd.read_csv('cadastro.csv', sep=';', encoding = 'ISO-8859-1')
cadastro = cadastro.drop(['Unnamed: 0'],axis=1)
cadastro

,CNPJ_FUNDO,DENOM_SOCIAL,SIT,CLASSE,VL_PATRIM_LIQ
0,00.000.684/0001-21,DEUTSCHE BANK FDO APLIC QUOTAS FDO INV FINANCEIRO - MAX,CANCELADA,NaN,NaN
1,00.000.731/0001-37,ITAMARITI CASH FUNDO APLICACAO QUOTAS FDOS INVESTIMENTO,CANCELADA,NaN,NaN
2,00.000.732/0001-81,FUNDO APLIC. QUOTAS DE F.I. SANTANDER CURTO PRAZO,CANCELADA,NaN,NaN
3,00.000.740/0001-28,FUNDO DE APLIC EM QUOTAS DE FUNDOS DE INV BMC 60 DIAS URV,CANCELADA,NaN,NaN
4,00.000.749/0001-39,BALANCE FUNDO APLICACAO QUOTAS FUNDO INVESTIMENTO FINANCEIRO,CANCELADA,NaN,NaN
...,...,...,...,...,...
74574,31.674.280/0001-47,MEDIA VENTURE ESG INCENTIVADO - FUNDO DE FINANCIAMENTO DA INDUSTRIA CINEMATOGRAFICA NACIONAL,CANCELADA,FUNCINE,0.00
74575,32.222.962/0001-81,Fundo de Financiamento da Indústria Cinematográfica Nacional de Minas Gerais,CANCELADA,FUNCINE,4652138.43
74576,48.879.897/0001-00,FUNDO DE FINANCIAMENTO DA INDÚSTRIA CINEMATOGRÁFICA NACIONAL - FUNCINE CEDRO LIGOCKI,FASE PRÉ-OPERACIONAL,FUNCINE,0.00
74577,52.714.528/0001-26,INVESTIMAGE 4 - FUNDO DE FINANCIAMENTO DA INDÚSTRIA CINEMATOGRÁFICA,EM FUNCIONAMENTO NORMAL,FUNCINE,731108.07


In [40]:
informes = busca_informes_diarios_cvm_por_periodo(data_inicio='2022-01', data_fim='2023-12')
fundos_ranking = ranking_fundos(informes, cadastro, top=40, minimo_de_cotistas=500, classe='acoes')

cotas = pd.DataFrame()
#fundos_destaque = pd.concat([melhores, piores])
fundos_destaque = pd.concat([fundos_ranking])

for cnpj in fundos_destaque.index:
  cotas = pd.concat([cotas, consulta_fundo(informes, cnpj)['cotas_normalizadas']], axis=1)

cotas.columns = fundos_destaque['Fundo de Investimento']
cotas.dropna(inplace=True)

fundos_ranking.to_csv(f"data/processed/fundos_ranking_24M_202201_202312.csv", sep = ";", encoding = 'ISO-8859-1')
cotas.to_csv(f"data/processed/cotas_24M_202201_202312.csv", sep = ";", encoding = 'ISO-8859-1')

In [41]:
informes_1M = busca_informes_diarios_cvm_por_periodo(data_inicio='2024-01', data_fim='2024-01')
fundos_ranking_1M = ranking_fundos(informes_1M, cadastro, top=40, minimo_de_cotistas=500, classe='acoes')

informes_2M = busca_informes_diarios_cvm_por_periodo(data_inicio='2024-02', data_fim='2024-02')
fundos_ranking_2M = ranking_fundos(informes_2M, cadastro, top=40, minimo_de_cotistas=500, classe='acoes')

informes_3M = busca_informes_diarios_cvm_por_periodo(data_inicio='2024-03', data_fim='2024-03')
fundos_ranking_3M = ranking_fundos(informes_3M, cadastro, top=40, minimo_de_cotistas=500, classe='acoes')

fundos_ranking_1M.to_csv(f"data/processed/fundos_ranking_1M_202401.csv", sep = ";", encoding = 'ISO-8859-1')
fundos_ranking_2M.to_csv(f"data/processed/fundos_ranking_2M_202402.csv", sep = ";", encoding = 'ISO-8859-1')
fundos_ranking_3M.to_csv(f"data/processed/fundos_ranking_3M_202403.csv", sep = ";", encoding = 'ISO-8859-1')

In [42]:
cotas = pd.DataFrame()
#fundos_destaque = pd.concat([melhores, piores])
fundos_destaque = pd.concat([fundos_ranking])

for cnpj in fundos_destaque.index:
  cotas = pd.concat([cotas, consulta_fundo(informes_1M, cnpj)['cotas_normalizadas']], axis=1)

cotas.columns = fundos_destaque['Fundo de Investimento']
cotas.dropna(inplace=True)
Retorno_1M = cotas.iloc[[-1]]-100

Retorno_1M.to_csv(f"data/processed/retorno_1M_202401.csv", sep = ";", encoding = 'ISO-8859-1')


In [43]:
cotas = pd.DataFrame()
#fundos_destaque = pd.concat([melhores, piores])
fundos_destaque = pd.concat([fundos_ranking])

for cnpj in fundos_destaque.index:
  cotas = pd.concat([cotas, consulta_fundo(informes_2M, cnpj)['cotas_normalizadas']], axis=1)

cotas.columns = fundos_destaque['Fundo de Investimento']
cotas.dropna(inplace=True)
Retorno_2M = cotas.iloc[[-1]]-100
Retorno_2M.to_csv(f"data/processed/retorno_2M_202402.csv", sep = ";", encoding = 'ISO-8859-1')


In [44]:
cotas = pd.DataFrame()
#fundos_destaque = pd.concat([melhores, piores])
fundos_destaque = pd.concat([fundos_ranking])

for cnpj in fundos_destaque.index:
  cotas = pd.concat([cotas, consulta_fundo(informes_3M, cnpj)['cotas_normalizadas']], axis=1)

cotas.columns = fundos_destaque['Fundo de Investimento']
cotas.dropna(inplace=True)
Retorno_3M = cotas.iloc[[-1]]-100
Retorno_3M.to_csv(f"data/processed/retorno_3M_202403.csv", sep = ";", encoding = 'ISO-8859-1')


Fundos com Shift

In [45]:
informes = busca_informes_diarios_cvm_por_periodo(data_inicio='2022-02', data_fim='2024-01')

cotas = pd.DataFrame()
#fundos_destaque = pd.concat([melhores, piores])]
fundos_destaque = pd.concat([fundos_ranking])

for cnpj in fundos_destaque.index:
  cotas = pd.concat([cotas, consulta_fundo(informes, cnpj)['cotas_normalizadas']], axis=1)

cotas.columns = fundos_destaque['Fundo de Investimento']
cotas.dropna(inplace=True)

cotas.to_csv(f"data/processed/cotas_24M_202202_202401_1M.csv", sep = ";", encoding = 'ISO-8859-1')

In [46]:
informes = busca_informes_diarios_cvm_por_periodo(data_inicio='2022-03', data_fim='2024-02')

cotas = pd.DataFrame()
#fundos_destaque = pd.concat([melhores, piores])
fundos_destaque = pd.concat([fundos_ranking])

for cnpj in fundos_destaque.index:
  cotas = pd.concat([cotas, consulta_fundo(informes, cnpj)['cotas_normalizadas']], axis=1)

cotas.columns = fundos_destaque['Fundo de Investimento']
cotas.dropna(inplace=True)

cotas.to_csv(f"data/processed/cotas_24M_202203_202402_2M.csv", sep = ";", encoding = 'ISO-8859-1')

In [47]:
informes = busca_informes_diarios_cvm_por_periodo(data_inicio='2022-04', data_fim='2024-03')

cotas = pd.DataFrame()
#fundos_destaque = pd.concat([melhores, piores])
fundos_destaque = pd.concat([fundos_ranking])

for cnpj in fundos_destaque.index:
  cotas = pd.concat([cotas, consulta_fundo(informes, cnpj)['cotas_normalizadas']], axis=1)

cotas.columns = fundos_destaque['Fundo de Investimento']
cotas.dropna(inplace=True)

cotas.to_csv(f"data/processed/cotas_24M_202204_202403_3M.csv", sep = ";", encoding = 'ISO-8859-1')

Retorno portfólio

In [90]:
Retorno_Portfolio = pd.DataFrame()

In [91]:
import numpy as np
import pandas as pd
from itertools import combinations

num_ativos = 4
Y = Retorno_1M.iloc[0].values

portfolios = list( combinations(range(0,len(Y)), num_ativos) )

# número de portfólio (número de pesos aleatórios gerados)
num_portfolios = len( portfolios )
 
# cria um array para manter os resultados
results = np.zeros(num_portfolios )
  
# cria o for loop para as simulações
for i in range(num_portfolios):
    # seleciona pesos aleatórios para os portfólios
    weights = np.ones(num_ativos)
    
    # verifica se os portfólios somam 1
    weights /= np.sum(weights)
    
    mu_carteira = Y[ [portfolios[i]] ]
    
    portfolio_return = np.sum(mu_carteira * weights)
    
    # armazena os valores no array
    results[i] = portfolio_return
    
# Converte o resultado para um df
Retorno_Portfolio['M1'] = results

In [92]:
num_ativos = 4
Y = Retorno_2M.iloc[0].values

portfolios = list( combinations(range(0,len(Y)), num_ativos) )

# número de portfólio (número de pesos aleatórios gerados)
num_portfolios = len( portfolios )
 
# cria um array para manter os resultados
results = np.zeros(num_portfolios )
  
# cria o for loop para as simulações
for i in range(num_portfolios):
    # seleciona pesos aleatórios para os portfólios
    weights = np.ones(num_ativos)
    
    # verifica se os portfólios somam 1
    weights /= np.sum(weights)
    
    mu_carteira = Y[ [portfolios[i]] ]
    
    portfolio_return = np.sum(mu_carteira * weights)
    
    # armazena os valores no array
    results[i] = portfolio_return
    
# Converte o resultado para um df
Retorno_Portfolio['M2'] = results

In [93]:
num_ativos = 4
Y = Retorno_3M.iloc[0].values

portfolios = list( combinations(range(0,len(Y)), num_ativos) )

# número de portfólio (número de pesos aleatórios gerados)
num_portfolios = len( portfolios )
 
# cria um array para manter os resultados
results = np.zeros(num_portfolios )
  
# cria o for loop para as simulações
for i in range(num_portfolios):
    # seleciona pesos aleatórios para os portfólios
    weights = np.ones(num_ativos)
    
    # verifica se os portfólios somam 1
    weights /= np.sum(weights)
    
    mu_carteira = Y[ [portfolios[i]] ]
    
    portfolio_return = np.sum(mu_carteira * weights)
    
    # armazena os valores no array
    results[i] = portfolio_return
    
# Converte o resultado para um df
Retorno_Portfolio['M3'] = results

In [94]:
Retorno_Portfolio

,1M,2M,3M
0,6.755085,-3.590942,-6.898343
1,6.736258,-3.584515,-6.906309
2,6.710001,-3.549377,-6.903332
3,6.703208,-3.520824,-6.807759
4,6.735155,-3.599805,-6.907377
...,...,...,...
91385,-1.558308,1.102327,-0.907728
91386,-2.005925,0.774445,-0.281898
91387,-1.550159,0.549408,-0.675102
91388,-1.927345,0.844237,-0.287616


In [95]:
Retorno_Portfolio.to_csv(f"data/processed/Retorno_Portfolio.csv", sep = ";", encoding = 'ISO-8859-1')